# Compute read co-occurrence data for phasing

In [1]:
%run "Header.ipynb"

In [24]:
import time
import pickle
import pysam
import skbio
from collections import defaultdict
from itertools import combinations
from linked_mutations_utils import find_mutated_positions

## For each read, identify all nucleotides aligned to mutated positions spanned by this read

In [25]:
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")

# This probably won't save a noticeable amount of memory, but humor me
i2n = "ACGT"
n2i = {"A": 0, "C": 1, "G": 2, "T": 3}

t1 = time.time()
for seq in SEQS:
    fasta = skbio.DNA.read("../seqs/{}.fasta".format(seq))
    
    # Identify all mutated positions in this genome up front to save time.
    print(f"Identifying mutated positions in genome {seq2name[seq]}...")
    mutated_positions = find_mutated_positions(seq)
    print(f"Found {len(mutated_positions):,} mutated positions in {seq2name[seq]}.")
    print("Going through these positions...")
    
    # This should already be implicitly sorted, I think, but the code below relies on mutated_positions being
    # in the exact same order as expected. So we may as well be paranoid.
    mutated_positions = sorted(mutated_positions)
    
    # Maps read name to another dict of mutated position -> aligned nucleotide (in A, C, G, T).
    # We build this up all at once so that we can take supplementary alignments of the same read into account.
    readname2mutpos2nt = defaultdict(dict)
    
    # Go through all linear alignments of each read to this genome...
    ts1 = time.time()
    for ai, aln in enumerate(bf.fetch(seq), 1):
        if ai % 1000 == 0:
            print(
                f"\tOn aln {ai:,} in seq {seq2name[seq]}. "
                f"Time spent on {seq2name[seq]} so far: {time.time() - ts1:,.2f} sec."
            )
        ap = aln.get_aligned_pairs(matches_only=True)
        
        # Iterating through the aligned pairs is expensive. Since read lengths are generally in the thousands
        # to tens of thousands of bp (which is much less than the > 1 million bp length of any bacterial genome),
        # we set things up so that we only iterate through the aligned pairs once. We maintain an integer, mpi,
        # that is a poor man's "pointer" to an index in mutated_positions.
        
        mpi = 0
        
        # Go through this aln's aligned pairs. As we see each pair, compare the pair's reference position
        # (refpos) to the mpi-th mutated position (herein referred to as "mutpos").
        #
        # If refpos >  mutpos, increment mpi until refpos <= mutpos (stopping as early as possible).
        # If refpos == mutpos, we have a match! Update readname2mutpos2ismutated[mutpos] based on
        #                      comparing the read to the reference at the aligned positions.
        # If refpos <  mutpos, continue to the next pair.
        
        readname = aln.query_name
        for pair in ap:
            
            refpos = pair[1]
            mutpos = mutated_positions[mpi]
            
            no_mutations_to_right_of_here = False
            
            # Increment mpi until we get to the next mutated position at or after the reference pos for this
            # aligned pair (or until we run out of mutated positions).
            while refpos > mutpos:
                mpi += 1
                if mpi < len(mutated_positions):
                    mutpos = mutated_positions[mpi]
                else:
                    no_mutations_to_right_of_here = True
                    break
            
            # I expect this should happen only for reads aligned near the right end of the genome.
            if no_mutations_to_right_of_here:
                break
            
            # If the next mutation occurs after this aligned pair, continue on to a later pair.
            if refpos < mutpos:
                continue
                
            # If we've made it here, refpos == mutpos!
            # (...unless I messed something up in how I designed this code.)
            if refpos != mutpos:
                raise ValueError("This should never happen!")
                
            # Finally, get the nucleotide aligned to this mutated position from this read.
            readpos = pair[0]
            # (Convert the nucleotide to an integer in the range [0, 3] using n2i)
            readval = n2i[aln.query_sequence[readpos]]
            
            # Record this specific "allele" for this read. We can use this to link alleles that co-occur
            # on the same read.
            readname2mutpos2nt[readname][mutpos] = readval
            
    with open(f"phasing-data/{seq}_readname2mutpos2nt.pickle", "wb") as dumpster:
        dumpster.write(pickle.dumps(readname2mutpos2nt))
        
print(f"Time taken: {time.time() - t1} sec.")

Identifying mutated positions in genome CAMP...
Found 283 mutated positions in CAMP.
Going through these positions...
	On aln 1000 in seq CAMP. Time spent on CAMP so far: 1.96 sec.
Identifying mutated positions in genome BACT1...
Found 23,971 mutated positions in BACT1.
Going through these positions...
	On aln 1000 in seq BACT1. Time spent on BACT1 so far: 1.92 sec.
Identifying mutated positions in genome BACT2...
Found 1,631 mutated positions in BACT2.
Going through these positions...
	On aln 1000 in seq BACT2. Time spent on BACT2 so far: 2.16 sec.
Time taken: 53.520060539245605 sec.


## Compute the numbers of reads supporting each observed combination of nucleotides at each pair of mutated positions spanned by at least _minSpan_ reads

TODO: Load the pickled dicts you just output, so this step can be done independently of the previous one.

We could use Hansel to store this data, but I opted to use a custom solution (for now, at least) for a few reasons:

1. Don't need anything fancy -- just need to store this, not use the probabilistic weighting stuff
2. I don't have time right now to learn Hansel's API (I've read through the docs and am still a bit confused)
3. I think we could probably use less storage (e.g. we only need to store one "triangle" of the matrix; as far as I can tell, Hansel treats H\[a, b, i, j\] as independent of H\[b, a, j, i\], which isn't desirable for haplotyping IMO

In [ ]:
#     # Now we've seen all alignments of each read, we can go through readname2mutpos2nt and compute co-occurrence
#     # information (and create a graph, plot stuff, etc.)
#     for readname in readname2mutpos2nt:
#         mutated_positions_covered_in_read = readname2mutpos2nt[readname].keys()
#         # Now that we've seen all mutated positions covered by this read, update pair information.
        
#         for (ii, jj) in combinations(mutated_positions_covered_in_read, 2):
            
#             # To make life easier, just sort the pair and save that as i and j.
#             # I *think* we could sort mutated_positions_covered_in_read and then combinations() should
#             # automatically generate sorted combinations, but I'm not sure if that is guaranteed -- so to
#             # reduce the probability of weird bugs we can just sort things here.
#             i, j = sorted([ii, jj])
            
#             # See if i and j are close enough to each other. There are two ways this can happen (these aren't
#             # necessarily mutually exclusive but in practice probs will be, depending on genome size and
#             # MAX_DIST_BTWN_LINKED_POSITIONS_NONINCLUSIVE)
#             #
#             # 1. i and j are close to each other without looping around the genome
#             #    (e.g. i = 15,000; j = 15,001)
#             #
#             # 2. i and j are close to each other when you loop around the genome
#             #    (e.g. genome length = 1,000,000; i = 0; j = 999,999)
#             #    This case is only allowed when seq2iscircular[seq] is True. (For edges that aren't circular --
#             #    e.g. edge 6104 [CAMP], as of writing, which is a linear edge within a circular component --
#             #    we don't allow this case to ever be True.)
            
#             # Case 1
#             close_enough_nolooping = (j - i) < MAX_DIST_BTWN_LINKED_POSITIONS_NONINCLUSIVE
            
#             # Case 2
#             if seq2iscircular[seq]:
#                 close_enough_looping = (seq2len[seq] + i - j) < MAX_DIST_BTWN_LINKED_POSITIONS_NONINCLUSIVE
#             else:
#                 close_enough_looping = False
                
#             if close_enough_nolooping or close_enough_looping:
#                 im = readname2mutpos2ismutated[readname][i]
#                 jm = readname2mutpos2ismutated[readname][j]
#                 if im:
#                     if jm:
#                         # Read supports mutations at both i and j
#                         pospair2groupcts[(i, j)][0] += 1
#                     else:
#                         # Read supports a mutation at i but not j
#                         pospair2groupcts[(i, j)][1] += 1
#                 else:
#                     if jm:
#                         # Read supports a mutation at j but not i
#                         pospair2groupcts[(i, j)][2] += 1
#                     else:
#                         # Read doesn't support mutations at either i or j
#                         pospair2groupcts[(i, j)][3] += 1

#     print(f"Finished going through reads in {seq2name[seq]}.")
    
#     # Write out pospair2json to a safe location, just because this is probably going to take a while
#     # and I don't want to risk losing this work.
#     #
#     # We use pickle instead of JSON because JSON can't handle tuples as the index of pospair2json:
#     # see https://stackoverflow.com/a/16439720.
#     # 
#     # We use the file suffix ".pickle" and "wb" based on the conventions described in
#     # https://stackoverflow.com/a/40433504 (...which in turn just reference the python docs).
    
#     with open(f"pospair2groupcts/{seq}_pospair2groupcts.pickle", "wb") as dumpster:
#         dumpster.write(pickle.dumps(pospair2groupcts))